Read me!
==================
`
This is the project 3 of the lab from the applied mathematics and statistics course.
The requirement is to using the linear regression to classify the wine problem.
`
- The source code is owned by Do Vuong Phuc (Phone number: 0707953475)
- Please do not copy the code!
- Created Date: 01/08/2021


In this project, we have 3 problems: A, B, C 
- A) Create a linear regression model that predict ${quality}=\sum_{i=1}^{11}{\theta_i.x_i}$ 
- B) Create 11 linear regression models and choose the best one (using Cross Validation) with the model $i^{th}$ is ${quality}=\theta.x_i$
- C) Create own model which is better


In [1]:
# Library and function
import pandas as pd
import numpy as np

# Finding the set of theta value for the optimize squared problem ||Ax-b||^2 min
def findTheta(A,b):
    A_t = np.transpose(A)
    P1=np.linalg.inv(A_t@A)
    P2=A_t@b
    return P1@P2

# Calculate the value of ||Ax-b||
def findNorm(A,b,x):
    return (np.linalg.norm(A@x-b))

# Train the model by the train set and test with the test set
def trainAndTest(train_set,test_set,y_name):
    train_y = train_set[y_name].to_frame().values
    test_y = test_set[y_name].to_frame().values
    
    train_field = train_set.copy()
    test_field = test_set.copy()
    del train_field[y_name]
    del test_field[y_name]
    
    theta = findTheta(train_field,train_y).values
    norm = findNorm(test_field,test_y,theta)
    return theta, norm

# Doing K-fold CV for the data with nFold size
def crossValidation(df,y_name,nFold):
    frame = df.copy()
    fold = np.array_split(frame,nFold)
    sNorm = 0
    listNormTheta = []
    
    for i in range(nFold):
        train_set = fold.copy()
        del train_set[i]
        test_set = fold[i]
        train_set = pd.concat(train_set)
        
        theta, norm = trainAndTest(train_set,test_set,y_name)
        listNormTheta.append((norm, theta))
        sNorm += norm
    selectedTheta = min(listNormTheta)[1]
    avgNorm = sNorm/nFold
    return selectedTheta, avgNorm

# Create a new data frame which on contains the field in listCol and y_name
def createNewFrame(df,listCol,y_name): 
    frame = df.copy()
    for col in frame.columns:
        if (col!=y_name and not(col in listCol)):
            del frame[col]
    return frame

In [2]:
# Read data
df = pd.read_csv('wine.csv',sep=';')

In [3]:
# Problem A: model y=sigma(theta_i.x_i)
theta, norm = trainAndTest(df,df,'quality')
print("Theta:\n",theta)
print("Norm:",norm)

Theta:
 [[ 5.92516137e-03]
 [-1.10803754e+00]
 [-2.63046284e-01]
 [ 1.53222831e-02]
 [-1.73050274e+00]
 [ 3.80141908e-03]
 [-3.89899869e-03]
 [ 4.33858768e+00]
 [-4.58535475e-01]
 [ 7.29718662e-01]
 [ 3.08858648e-01]]
Norm: 22.124345965349157


In [4]:
# Problem B: models y_i=theta_i.x_i
result = []
for col in df.columns:        
    if (col!='quality'):
        frame = createNewFrame(df,[col],'quality')
        selectedTheta, avgNorm = crossValidation(frame,'quality',10)
        result.append((avgNorm,selectedTheta,col))
        
bestResult = min(result)
print('= Best result =')
print('Norm: ',bestResult[0])
print('Theta:',bestResult[1])
print('Attribute:',bestResult[2])

= Best result =
Norm:  7.918652557961548
Theta: [[0.54298689]]
Attribute: alcohol


In [5]:
# Problem C: define your own model

def testAllCase(result=[],ls=[],k=0):
    if (k==len(df.columns)-1):
        frame = createNewFrame(df,ls,'quality')
        selectedTheta, avgNorm = crossValidation(frame,'quality',10)
        t = (avgNorm,selectedTheta,ls)
        result.append(t)
    else:
        ls = ls.copy()
        for i in range(2):
            if (i==1):
                ls.append(df.columns[k])
            testAllCase(result,ls,k+1)

listResult = []
testAllCase(listResult)
bestResult = min(listResult)
print("= Best result =")
print("Norm: ",bestResult[0])
print("Theta:")
print(bestResult[1])
print("Attribute")
print(bestResult[2])

= Best result =
Norm:  7.09667876204013
Theta:
[[-1.05817351e+00]
 [-2.65570255e-01]
 [-1.68571405e+00]
 [-3.43210474e-03]
 [ 4.29851787e+00]
 [-4.60521429e-01]
 [ 7.04482782e-01]
 [ 3.24614128e-01]]
Attribute
['volatile acidity', 'citric acid', 'chlorides', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']


In [7]:
#Testing problem A with cross validation

theta, norm = crossValidation(df,'quality',10)
print("Theta:\n",theta)
print("Norm:",norm)

Theta:
 [[ 3.83862334e-03]
 [-1.05270482e+00]
 [-2.79617426e-01]
 [ 1.43406488e-02]
 [-1.65757834e+00]
 [ 2.83437290e-03]
 [-4.09579872e-03]
 [ 4.21256719e+00]
 [-4.44796104e-01]
 [ 7.02641261e-01]
 [ 3.19940378e-01]]
Norm: 7.126524340809087
